In [3]:
import random
import csv
import os
import os.path
import shutil
import cv2

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split, ConcatDataset
import torchvision.models as models
import tqdm.notebook as tq

import pandas as pd
import h5py
import math


In [2]:
############
# Download Data from Google Drive
############
%cd /content
!gdown https://drive.google.com/uc?id=1olKESqgAm5GeAmuRC21BpTBkxFrgmUIp
!unzip /content/Data

print("Finished Downloading data")

Streaming output truncated to the last 5000 lines.
  inflating: Data/Covid/18260.png    
  inflating: Data/Covid/18267.png    
  inflating: Data/Covid/18278.png    
  inflating: Data/Covid/18280.png    
  inflating: Data/Covid/18308.png    
  inflating: Data/Covid/18310.png    
  inflating: Data/Covid/18315.png    
  inflating: Data/Covid/18334.png    
  inflating: Data/Covid/18335.png    
  inflating: Data/Covid/18339.png    
  inflating: Data/Covid/18348.png    
  inflating: Data/Covid/18362.png    
  inflating: Data/Covid/18368.png    
  inflating: Data/Covid/18370.png    
  inflating: Data/Covid/18378.png    
  inflating: Data/Covid/18389.png    
  inflating: Data/Covid/18410.png    
  inflating: Data/Covid/18420.png    
  inflating: Data/Covid/18453.png    
  inflating: Data/Covid/18479.png    
  inflating: Data/Covid/18480.png    
  inflating: Data/Covid/18489.png    
  inflating: Data/Covid/18498.png    
  inflating: Data/Covid/18505.png    
  inflating: Data/Covid/18524.png    

In [4]:
###################
# Parameters we use for the test
###################
params = {
  "batch": 32,
  "l1Tol2Features": 64,
  "l2tol3Features": 32,
  "epochs": 7,
  "lr": 0.001,
  "wtDecay": 0.4,
  "frozenChildren": 5,
  'momentum': 0.5 
}

In [6]:
#######################
# Do Cross-validation in 5 layers
#######################

#####################
# Generate 5 folders with labels
#####################
def datasetGen(k=5):
  # csv label fields 
  fields = ['Name', 'label']

  # First we check if we have the directory created already
  # If yes, delete it and create it again
  # This is to avoid error when creating the lables
  %cd /content/
  if os.path.exists('Dataset'):
    shutil.rmtree('Dataset')
  os.makedirs('Dataset')

  # Partition into five different subsets  
  %cd Dataset/
  for i in range(k):
    os.makedirs(str(i))
    with open(os.path.join(str(i), "label.csv"), 'w+') as csvfile:
      csvwriter = csv.writer(csvfile) 
      csvwriter.writerow(fields)

########################
# Helper function to write label to csv
#######################
def csvWriteData(data, set):  
  with open(os.path.join('/content/Dataset', str(set), "label.csv"), 'a') as csvfile:   
      csvwriter = csv.writer(csvfile)  
      csvwriter.writerow(data)   

########################
# Helper function to sort images
#######################
def imgSort(dir, maxImg, index, label): 
  counter = [0,0,0,0,0]
  for dirpath, dirnames, filenames in os.walk(dir):
    for filename in [f for f in filenames if f.endswith(".png")]: 

      # Read image
      img = cv2.imread(os.path.join(dirpath, filename), cv2.IMREAD_UNCHANGED)
      if img is None:
        continue

      # Get Random Number
      subAssign = random.uniform(0, 5)

      # Folder 0
      if subAssign < 1 and counter[0] <= maxImg:
        counter[0] = counter[0] + 1
        csvWriteData([index, label], 0)
        cv2.imwrite(os.path.join('/content/Dataset/0', str(index)+'.png'), img)

      # Folder 1
      elif subAssign < 2 and counter[1] <= maxImg:
        counter[1] = counter[1] + 1
        csvWriteData([index, label], 1)
        cv2.imwrite(os.path.join('/content/Dataset/1', str(index)+'.png'), img)

      # Folder 2
      elif subAssign < 3 and counter[2] <= maxImg:
        counter[2] = counter[2] + 1
        csvWriteData([index,  label], 2)
        cv2.imwrite(os.path.join('/content/Dataset/2', str(index)+'.png'), img)
      
      # Folder 3
      elif subAssign < 4 and counter[3] <= maxImg:
        counter[3] = counter[3] + 1
        csvWriteData([index, label], 3)
        cv2.imwrite(os.path.join('/content/Dataset/3', str(index)+'.png'), img)

      # Folder 4
      else:
        counter[4] = counter[4] + 1
        csvWriteData([index, label], 4)
        cv2.imwrite(os.path.join('/content/Dataset/4', str(index)+'.png'), img)

      index += 1
  return index

# Create dataset structure
datasetGen(5)

# Distribute images
print("sorting images")
index = 1
index = imgSort('/content/Data/Covid', 2163, index, 0)
index = imgSort('/content/Data/Healthy', 909, index, 1)
index = imgSort('/content/Data/Others', 1497, index, 2)
print("done!")
print(index)


/content
/content/Dataset
sorting images
done!
19031


In [7]:
############################
# Load our DataSet
############################
class Data:
    ########################
    # Constructor, initialize variables
    ########################
    def __init__(self, root):
        self.ROOT = root
        self.images, self.mn, self.std = self.read_images(self.ROOT)
        self.labels = self.read_labels(self.ROOT)
        
        # ======================================================================================
        # Composes several transforms together. Basically, do a bunch of transformation
        # ======================================================================================
        self.transform = transforms.Compose([
            # Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor.                                    
            transforms.ToTensor(),
            # ===============================================
            # Normalize a tensor image with mean and standard deviation.
            # ===============================================            
            transforms.Normalize(self.mn, self.std)      
            ])
        
    ##################
    # Length of dataset
    ##################
    def __len__(self):
        # Return number of points in the dataset
        return len(self.images)

    ##################
    # Get Images and Label
    # Here we have to return the item requested by `idx`. The PyTorch DataLoader class will use this method to make an iterable for training/validation loop.
    ##################
    def __getitem__(self, idx):
        # Encounter some errors, img is a 1 channel, use a np.concatenate to make 3 channels
        img = self.transform(self.images[idx])
        img = np.concatenate((img, img, img), 0)
        label = self.labels[idx]
        return img, label
  
    # Read Images
    def read_images(self, path:str) -> list:
        imgs = []
        mn = 0
        std = 0
        count = 0
        for filename in sorted([f for f in os.listdir(path) if f.endswith(".png")], key=lambda f : int(f[:-4])): 
          img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_UNCHANGED)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          imgs.append(img)
          mn += np.mean(img)
          std += np.var(img)
          count += 1
        mn /= count*255
        std = math.sqrt(std / (count)) / 255
        return imgs, mn, std
      
    # Read Labels
    def read_labels(self, path:str) -> list:
        labels = pd.read_csv(os.path.join(path, 'label.csv'))
        return labels.label

###################
# Create my 5 sets/cross-validation
###################
# Create datasets
print("Creating my 5 sets")
split = []
for i in range(5):
  split.append(Data(f'/content/Dataset/{i}'))

print("Finished Creating")
print("Size:" + str(len(split)))


Creating my 5 sets
Finished Creating
Size:5


In [ ]:
#########################
# ML Network
#########################
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: [Transfer learning with pre-trained ResNet-50] 
        # 1) Define how many first layers of convolutoinal neural network (CNN) feature extractor in ResNet-50 to be "frozen" and 
        # 2) design your own fully-connected network (FCN) classifier.
        
        # 1) You will only refine last several layers of CNN feature extractor in ResNet-50 that mainly relate to high-level vision task. Determine how many first layers of ResNet-50 should be frozen to achieve best performances. Commented codes below will help you understand the architecture, i.e., "children", of ResNet-50.
        # 2) Design your own FCN classifier. Here I provide a sample of two-layer FCN.
        # Refer to PyTorch documentations of torch.nn to pick your layers. (https://pytorch.org/docs/stable/nn.html)
        # Some common Choices are: Linear, ReLU, Dropout, MaxPool2d, AvgPool2d
        # If you have many layers, consider using nn.Sequential() to simplify your code
        
        # Load pretrained ResNet-50
        self.model_resnet = models.resnet50(pretrained=True)
        
        # The code below can show children of ResNet-50
        #child_counter = 0
        #for child in model.children():
        #    print(" child", child_counter, "is -")
        #    print(child)
        #    child_counter += 1
        
        # ==================================================================
        # [Transfer learning with pre-trained ResNet-50] 
        # ==================================================================

        # ==================================================================
        # Determine how many first layers of ResNet-50 to freeze
        # ==================================================================
        '''
          How to decide how many layers to freeze?
        '''

        child_counter = 0
        for child in self.model_resnet.children():
            if child_counter < 5:
                for param in child.parameters():
                    param.requires_grad = False
            elif child_counter == 5:
                children_of_child_counter = 0
                for children_of_child in child.children():
                    if children_of_child_counter < 3:
                        for param in children_of_child.parameters():
                            param.requires_grad = False
                    else:
                        children_of_child_counter += 1
            else:
                print("child ",child_counter," was not frozen")
            child_counter += 1
        
        # Set ResNet-50's FCN as an identity mapping
        num_fc_in = self.model_resnet.fc.in_features
        self.model_resnet.fc = nn.Identity()
        
        # ==================================================================
        # Design your own FCN
        # ==================================================================
        # Increase dimension and then decreate back
        self.fc1 = nn.Linear(num_fc_in, params['l1Tol2Features'], bias = 3) # from input of size num_fc_in to output of size ?
        self.bn1 = nn.BatchNorm1d(params['l1Tol2Features'])
        self.fc2 = nn.Linear(params['l1Tol2Features'], params['l2tol3Features'], bias = 3) # from hidden layer to 3 class scores
        self.bn2 = nn.BatchNorm1d(params['l2tol3Features'])
        self.fc3 = nn.Linear(params['l2tol3Features'], 3, bias = 3)

    def forward(self,x):
        # ==================================================================
        # Design your own network, implement forward pass here
        # ==================================================================\
      relu = nn.ReLU()
      
      with torch.no_grad():
          features = self.model_resnet(x)
  
      x = self.fc1(features)
      x = relu(x)
      x = self.bn1(x)
      x = self.fc2(x)
      x = relu(x)
      x = self.bn2(x)
      x = self.fc3(x)

      return x

def train(model, loader, test, num_epoch, criterion, optimizer): # Train the model
    print("Start training...")
    model.train() # Set the model to training mode

    ########################
    # Run epochs with this parameters
    ########################
    for i in range(num_epoch):
        running_loss = []
        testing_loss = []
        for batch, label in tqdm(loader, position=0, leave=True):
            batch = batch.to(device)
            label = label.to(device)
            optimizer.zero_grad() # Clear gradients from the previous iteration
            pred = model(batch) # This will call Network.forward() that you implement
            loss = criterion(pred, label) # Calculate the loss
            running_loss.append(loss.item())
            loss.backward() # Backprop gradients to all tensors in the network
            optimizer.step() # Update trainable weights

        ####################
        # Now, we don't update the gradient since we don't want to bias our testing data
        ####################
        with torch.no_grad():
          for batch, label in tq.tqdm(test):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            loss = criterion(pred, label)
            testing_loss.append(loss.item())

        # Print loss
        print("Epoch {} training loss:{}".format(i+1,np.mean(running_loss))) 
        print("Epoch {} testing loss:{}".format(i+1,np.mean(testing_loss))) 

        saveData.append((i+1, np.mean(running_loss)))
        saveData.append((i+1, np.mean(testing_loss)))

    print("Done!")

def evaluate(model, loader): # Evaluate accuracy on validation / test set
    model.eval() # Set the model to evaluation mode
    correct = 0
    with torch.no_grad(): # Do not calculate grident to speed up computation
        for batch, label in tqdm(loader, position=0, leave=True):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            correct += (torch.argmax(pred,dim=1)==label).sum().item()

    acc = correct/len(loader.dataset)
    print("Evaluation accuracy: {}".format(acc))
    return acc

In [9]:
########################
# Cross-Validation Load Data into 5 sections
########################
# First batch train/test
b1Train = DataLoader(ConcatDataset([split[1], split[2], split[3], split[4]]), batch_size=params["batch"], shuffle=True, drop_last=True)
b1Test = DataLoader(split[0], batch_size=params["batch"], shuffle=True, drop_last=True)

# Second batch train/test
b2Train = DataLoader(ConcatDataset([split[0], split[2], split[3], split[4]]), batch_size=params["batch"], shuffle=True, drop_last=True)
b2Test = DataLoader(split[1], batch_size=params["batch"], shuffle=True, drop_last=True)

# Third batch train/test
b3Train = DataLoader(ConcatDataset([split[0], split[1], split[3], split[4]]), batch_size=params["batch"], shuffle=True, drop_last=True)
b3Test = DataLoader(split[2], batch_size=params["batch"], shuffle=True, drop_last=True)

# Fourth batch train/test
b4Train = DataLoader(ConcatDataset([split[0], split[1], split[2], split[4]]), batch_size=params["batch"], shuffle=True, drop_last=True)
b4Test = DataLoader(split[3], batch_size=params["batch"], shuffle=True, drop_last=True)

# Fifth batch train/test
b5Train = DataLoader(ConcatDataset([split[0], split[1], split[2], split[3]]), batch_size=params["batch"], shuffle=True, drop_last=True)
b5Test = DataLoader(split[4], batch_size=params["batch"], shuffle=True, drop_last=True)

# Store train/test pairs in list
pairs = [(b1Train, b1Test), (b2Train, b2Test), (b3Train, b3Test), (b4Train, b4Test), (b5Train, b5Test)]


In [11]:
########################
# Cross-Validation Traind data
########################
batchAccuracies = []
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
num_epochs = 6
criterion = nn.CrossEntropyLoss()


saveData = []

for pair in pairs:
  
  model= Network().to(device)
  #optimizer = optim.Adadelta(model.parameters(), lr=0.01)
  optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.35)
  # Train on new batch
  train(model, pair[0], pair[1], num_epochs, criterion, optimizer)
  print("Evaluate on test set")

  # Save eval result
  batchAccuracies.append(evaluate(model, pair[1]))

  # Deleted model
  del model
print(f'Evaluation Accuracy for each batch: {batchAccuracies}')
print(f'Evaluation Accuracy for each batch: {batchAccuracies}')
print(f'Average accuracy: {sum(batchAccuracies) / len(batchAccuracies)}')


  0%|          | 2/478 [00:00<00:37, 12.84it/s]

child  6  was not frozen
child  7  was not frozen
child  8  was not frozen
child  9  was not frozen
Start training...


100%|██████████| 478/478 [00:35<00:00, 13.51it/s]


  0%|          | 2/478 [00:00<00:37, 12.79it/s]


Epoch 1 training loss:0.6235469287284747
Epoch 1 testing loss:0.5539414664280826


...

100%|██████████| 476/476 [00:35<00:00, 13.37it/s]


  2%|▏         | 2/118 [00:00<00:08, 14.14it/s]


Epoch 6 training loss:0.36100643499120444
Epoch 6 testing loss:0.42926085411997167
Done!
Evaluate on test set


100%|██████████| 118/118 [00:08<00:00, 14.50it/s]

Evaluation accuracy: 0.8496300211416491
Evaluation Accuracy for each batch: [0.8483221476510067, 0.8520540821632865, 0.8614864864864865, 0.8547164881107917, 0.8496300211416491]
Evaluation Accuracy for each batch: [0.8483221476510067, 0.8520540821632865, 0.8614864864864865, 0.8547164881107917, 0.8496300211416491]
Average accuracy: 0.853241845110644


In [36]:
with open('/content/params.csv', 'w+') as store:
  storeWrite = csv.writer(store, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  #storeWrite.writerow(["tnMean", "tstMean", "tnStd", "tstStd", "batch", "l1Tol2Features", "epochs", "lr", "wtDecay", "frozenChildren"])
  for key, val in params.items():
    storeWrite.writerow([key, val])
  storeWrite.writerow(['epochNum', 'loss'])
  storeWrite.writerows(saveData)
  storeWrite.writerow(['Accuracy', batchAccuracies])

In [ ]:
#  7epochs looks the best...